# Análise exploratória de dados - GamersClub CS:GO

In [16]:
import pandas as pd
import plotly.express as px
import datetime as dt

## Parte 0: qualidade dos dados

In [27]:
lobby_players = pd.read_csv('data/tb_lobby_stats_player.csv')

In [36]:
# count lines
print('Number of lines: ', len(lobby_players))

Number of lines:  184152


In [37]:
lobby_players['idLobbyGame'].value_counts().sort_values(ascending=False).head(10)

58891    5
60420    4
17782    4
17622    4
71305    4
93223    4
17777    4
4508     4
17644    3
31686    3
Name: idLobbyGame, dtype: int64

In [38]:
lobby_players['idRoom'].value_counts().sort_values(ascending=False).head(10)

17861    3
17801    3
17818    3
17817    3
17858    3
17876    3
58765    3
24323    3
17887    3
17889    3
Name: idRoom, dtype: int64

## Parte 1: players

In [2]:
players = pd.read_csv('data/tb_players.csv')

In [14]:
players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2716 entries, 0 to 2715
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   idPlayer        2716 non-null   int64 
 1   flFacebook      2716 non-null   int64 
 2   flTwitter       2716 non-null   int64 
 3   flTwitch        2716 non-null   int64 
 4   descCountry     2716 non-null   object
 5   dtBirth         2008 non-null   object
 6   dtRegistration  2716 non-null   object
dtypes: int64(4), object(3)
memory usage: 148.7+ KB


In [3]:
players.head(10)

,idPlayer,flFacebook,flTwitter,flTwitch,descCountry,dtBirth,dtRegistration
0,1,0,0,0,cl,NaN,2021-03-19 21:31:39
1,2,0,0,0,br,NaN,2020-05-06 19:28:29
2,3,0,0,0,br,NaN,2020-01-28 13:00:38
3,4,0,0,0,br,NaN,2017-05-31 16:13:51
4,5,0,0,0,ar,NaN,2021-02-08 11:57:21
5,6,0,0,0,br,2002-03-23 00:00:00,2019-01-09 19:24:27
6,7,0,0,0,br,NaN,2020-04-24 22:53:50
7,8,0,0,0,br,NaN,2021-06-26 14:47:57
8,9,0,0,0,ar,2000-05-03 00:00:00,2016-05-22 15:12:48
9,10,0,0,0,br,2005-04-15 00:00:00,2018-02-28 11:29:11


In [4]:
print(players['flFacebook'].value_counts())
print(players['flTwitter'].value_counts())
print(players['flTwitch'].value_counts())

0    2653
1      63
Name: flFacebook, dtype: int64
0    2672
1      44
Name: flTwitter, dtype: int64
0    2663
1      53
Name: flTwitch, dtype: int64


Explicação das colunas:
* ``idPlayer``: id do player
* ``flFacebook``, ``flTwitter``, ``flTwitch``: se o jogador sincronizou (1) ou não (0) sua conta com cada uma das plataformas
* ``descCountry``: sigla de dois dígitos indicando o país do jogador
* ``dtRegistration``: data de ingresso do jogador na plataforma GamersClub

In [5]:
players['descCountry'].isna().sum()

0

Ou seja, todos os players tem um país.

In [6]:
countries_count = players['descCountry'].value_counts()

In [13]:
# Plot to donut chart
fig = px.pie(countries_count, values=countries_count.values, names=countries_count.index, title='Países')
fig.update_traces(textposition='inside', textinfo='percent+label')
# add hole
fig.update_traces(hole=.6, hoverinfo="label+percent+name")
fig.show()


In [17]:
# Calculate the age of the players
players['age'] = (dt.datetime.now() - pd.to_datetime(players['dtBirth'])).astype('<m8[Y]')

In [23]:
# Plot to bar graph (cap between bars)
fig = px.histogram(players, x='age', title='Idade dos jogadores')
fig.update_layout(bargap=0.2)
fig.show()

## Ideias

* Roubar [desse cara](https://www.kaggle.com/code/gustavolatorresilva/eda-gc)

---

* Win rate - nível / mapa
* ou ainda, relação: quais mapas são mais jogados por nível
* kda por nível -> interessante ver distribuição
* survived por nível -> ver se níveis mais altos tendem a guardar mais
* flash assist por nível
* granada em geral por nível
* granada por mapa -> quais mapas são táticos
* hs por nível
* tiros por nível
* qtd. rounds por nível (ver se level mais baixo tende a ter mais stomp) - tentar ver se partidas de nível mais alto tendem a ser mais equilibradas

---

* distribuição de rating 2.0 (tem que calcular - pegar fórmula)
* ADR por player -- vitória ou derrota por exemplo

---
* cruzar stats com derrota a vitória: ADR e rating acho que são bons

---

* fazer algumas descritivas: mapas, país de player, idade...
* dá pra tirar umas medidas de idade também (média, mediana, desvio, distribuição, etc.)

O que dá pra fazer com isso?

* distribuição / assimetria
* medidas que ele ensinou (a porra toda, igual o Voltera fez) -- a porra toda, mas n precisa ser de tudo. Ou seja, fazemos as relações primeiro, depois tiramos as medidas do que for mais importante, de preferência já agrupadas por players. Ex.: média de kill por player.
* regressão com o que fizer sentido
* correlação com o que puder
* enfim, fazer as coisas que ele ensinou + algo a mais só pra humilhar a estatística (ex.: algum fit/regressão pica)

In [45]:
lobby_players['qtTk'].sort_values(ascending=False).head(10)

162076    12.0
22986      5.0
65498      4.0
70875      4.0
18990      3.0
27468      3.0
49793      3.0
47274      3.0
161430     3.0
38089      3.0
Name: qtTk, dtype: float64